Link to colab: https://colab.research.google.com/drive/1IHSVKBQujBfJ09InG2-rYW2qagkB5JrE?usp=sharing

In [ ]:
!wget -O chess_data_ccrl.parquet 'https://drive.google.com/u/0/uc?id=1rJjSsnvxxbifdWs1yldLsCPwgepQmdXx&confirm=t'

!pip install chess

--2023-05-09 12:48:51--  https://drive.google.com/u/0/uc?id=1rJjSsnvxxbifdWs1yldLsCPwgepQmdXx&confirm=t
Resolving drive.google.com (drive.google.com)... 173.194.213.102, 173.194.213.139, 173.194.213.113, ...
Connecting to drive.google.com (drive.google.com)|173.194.213.102|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.google.com/uc?id=1rJjSsnvxxbifdWs1yldLsCPwgepQmdXx&confirm=t [following]
--2023-05-09 12:48:51--  https://drive.google.com/uc?id=1rJjSsnvxxbifdWs1yldLsCPwgepQmdXx&confirm=t
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-bo-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/f3t33t6i11ntgq8e7rsjd37dsbh7b337/1683636525000/18334911380517025247/*/1rJjSsnvxxbifdWs1yldLsCPwgepQmdXx?uuid=b8d0bc66-bf3a-478f-83dd-9668ccc38fd9 [following]
--2023-05-09 12:48:51--  https://doc-0s-bo-docs.googleusercontent.com/docs/securesc/ha0ro937g

In [ ]:
#@title Setup - Get ChessGNN repo (wait until session restarts)
def get_repo_and_install():
    import subprocess

    assert subprocess.getstatusoutput(f"git clone --depth=1 https://github.com/marcinz99/ChessGNN.git")[0] == 0
    assert subprocess.getstatusoutput(f"pip install -e ChessGNN/")[0] == 0
    exit()

get_repo_and_install()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import chess
import gc
import itertools
from chessgnn.chess_utils.data_preparation import (
    parse_and_replay,
    states_all_to_moves_dataframe,
    calculate_tracking,
    concat_initial_position,
    concat_shifted_features,
    generate_boards_daraframe,
    games_consistency_check,
)

pd.set_option('display.max_columns', None)
plt.rcParams['figure.dpi'] = 72

In [ ]:
df_ccrl = pd.read_parquet("chess_data_ccrl.parquet")
df_ccrl

,result,len,welo,belo,game
0,D,64,3388,3248,W1.e4 B1.c6 W2.d4 B2.d5 W3.e5 B3.Bf5 W4.Nf3 B4...
1,W,144,3249,3112,W1.e4 B1.c5 W2.Nf3 B2.Nc6 W3.Bb5 B3.g6 W4.Bxc6...
2,D,122,3188,3241,W1.e4 B1.e5 W2.Nf3 B2.Nc6 W3.Bb5 B3.a6 W4.Ba4 ...
3,W,149,3069,3054,W1.e4 B1.e5 W2.Nf3 B2.Nc6 W3.Bb5 B3.a6 W4.Ba4 ...
4,D,98,3457,3446,W1.d4 B1.d5 W2.c4 B2.dxc4 W3.Nf3 B3.Nf6 W4.e3 ...
...,...,...,...,...,...
576215,W,88,3168,3092,W1.d4 B1.Nf6 W2.c4 B2.e6 W3.Nc3 B3.Bb4 W4.Qc2 ...
576216,B,102,3192,3249,W1.c4 B1.e6 W2.g3 B2.Nf6 W3.Bg2 B3.d5 W4.Nf3 B...
576217,D,123,3184,3136,W1.c4 B1.Nf6 W2.g3 B2.c6 W3.Bg2 B3.d5 W4.Nf3 B...
576218,W,168,3195,3060,W1.c4 B1.e5 W2.g3 B2.c6 W3.b3 B3.Nf6 W4.Bb2 B4...


# Step-by-step definition

In [ ]:
states_all = parse_and_replay(games_df=df_ccrl, n_from=0, n_to=500)
states_all[:2]

Game: 500


[(0,
  64,
  'D',
  'e4',
  'e2e4',
  'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1',
  array([ 7, 24,  7, 22,  2, 19,  2, 17, 16, 24, 15, 23, 14, 22, 13, 21, 12,
         20, 11, 19, 10, 18,  9, 17, 16, 32, 15, 31, 14, 30, 13, 29, 12, 28,
         11, 27, 10, 26,  9, 25], dtype=uint8),
  array([], dtype=uint8)),
 (0,
  64,
  'D',
  'c6',
  'c7c6',
  'rnbqkbnr/pp1ppppp/2p5/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2',
  array([63, 48, 63, 46, 63, 53, 62, 17, 62, 26, 62, 35, 62, 44, 62, 53, 61,
         53, 60, 32, 60, 39, 60, 46, 60, 53, 58, 43, 58, 41, 37, 29, 56, 48,
         55, 47, 54, 46, 52, 44, 51, 43, 50, 42, 49, 41, 56, 40, 55, 39, 54,
         38, 52, 36, 51, 35, 50, 34, 49, 33], dtype=uint8),
  array([], dtype=uint8))]

In [ ]:
moves = states_all_to_moves_dataframe(states_all)
moves.head(5)

,game_index,total_len,final_result,latest_move_san,latest_move_uci,possible_moves,possible_captures,latest_move,piece_placement,who_moves_next,halfmove_clock,fullmove_number,ply_counter,state_repetition,lm_capture,lm_check,lm_mate,lm_kingside_castling,lm_queenside_castling,lm_promotion,en_passant_target,K_castling_ability,Q_castling_ability,k_castling_ability,q_castling_ability,place_from,place_to,promoted_to,lm_piece_type,en_passant_target_prev,lm_en_passant
0,0,64,D,e4,e2e4,"[7, 24, 7, 22, 2, 19, 2, 17, 16, 24, 15, 23, 1...",[],"[53, 37]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",b,0,1,1,0,False,False,False,False,False,False,0,True,True,True,True,53,37,,6,0,False
1,0,64,D,c6,c7c6,"[63, 48, 63, 46, 63, 53, 62, 17, 62, 26, 62, 3...",[],"[11, 19]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 0, 6, 6, 6, 6, ...",w,0,2,2,0,False,False,False,False,False,False,0,True,True,True,True,11,19,,6,0,False
2,0,64,D,d4,d2d4,"[7, 24, 7, 22, 4, 11, 4, 18, 4, 25, 2, 17, 16,...",[],"[52, 36]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 0, 6, 6, 6, 6, ...",b,0,2,3,0,False,False,False,False,False,False,0,True,True,True,True,52,36,,6,0,False
3,0,64,D,d5,d7d5,"[63, 48, 63, 46, 63, 53, 62, 17, 62, 26, 62, 3...","[37, 28]","[12, 28]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 0, 0, 6, 6, 6, ...",w,0,3,4,0,False,False,False,False,False,False,0,True,True,True,True,12,28,,6,0,False
4,0,64,D,e5,e4e5,"[7, 24, 7, 22, 5, 12, 4, 12, 4, 11, 4, 20, 4, ...",[],"[37, 29]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 0, 0, 6, 6, 6, ...",b,0,3,5,0,False,False,False,False,False,False,0,True,True,True,True,37,29,,6,0,False


In [ ]:
moves = calculate_tracking(moves)
moves.head(5)

Game: 500


,game_index,total_len,final_result,latest_move_san,latest_move_uci,possible_moves,possible_captures,latest_move,piece_placement,who_moves_next,halfmove_clock,fullmove_number,ply_counter,state_repetition,lm_capture,lm_check,lm_mate,lm_kingside_castling,lm_queenside_castling,lm_promotion,en_passant_target,K_castling_ability,Q_castling_ability,k_castling_ability,q_castling_ability,place_from,place_to,promoted_to,lm_piece_type,en_passant_target_prev,lm_en_passant,piece_tracking,piece_typing,lm_idx_moved,lm_idx_captured
0,0,64,D,e4,e2e4,"[7, 24, 7, 22, 2, 19, 2, 17, 16, 24, 15, 23, 1...",[],"[53, 37]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",b,0,1,1,0,False,False,False,False,False,False,0,True,True,True,True,53,37,,6,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[20],[]
1,0,64,D,c6,c7c6,"[63, 48, 63, 46, 63, 53, 62, 17, 62, 26, 62, 3...",[],"[11, 19]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 0, 6, 6, 6, 6, ...",w,0,2,2,0,False,False,False,False,False,False,0,True,True,True,True,11,19,,6,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 19, 12, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[10],[]
2,0,64,D,d4,d2d4,"[7, 24, 7, 22, 4, 11, 4, 18, 4, 25, 2, 17, 16,...",[],"[52, 36]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 0, 6, 6, 6, 6, ...",b,0,2,3,0,False,False,False,False,False,False,0,True,True,True,True,52,36,,6,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 19, 12, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[19],[]
3,0,64,D,d5,d7d5,"[63, 48, 63, 46, 63, 53, 62, 17, 62, 26, 62, 3...","[37, 28]","[12, 28]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 0, 0, 6, 6, 6, ...",w,0,3,4,0,False,False,False,False,False,False,0,True,True,True,True,12,28,,6,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 19, 28, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[11],[]
4,0,64,D,e5,e4e5,"[7, 24, 7, 22, 5, 12, 4, 12, 4, 11, 4, 20, 4, ...",[],"[37, 29]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 0, 0, 6, 6, 6, ...",b,0,3,5,0,False,False,False,False,False,False,0,True,True,True,True,37,29,,6,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 19, 28, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[20],[]


In [ ]:
moves = concat_initial_position(moves)
moves.head(5)

,game_index,total_len,final_result,latest_move_san,latest_move_uci,possible_moves,possible_captures,latest_move,piece_placement,who_moves_next,halfmove_clock,fullmove_number,ply_counter,state_repetition,lm_capture,lm_check,lm_mate,lm_kingside_castling,lm_queenside_castling,lm_promotion,en_passant_target,K_castling_ability,Q_castling_ability,k_castling_ability,q_castling_ability,place_from,place_to,promoted_to,lm_piece_type,en_passant_target_prev,lm_en_passant,piece_tracking,piece_typing,lm_idx_moved,lm_idx_captured,plies_till_end
0,0,64,D,--,----,"[63, 48, 63, 46, 58, 43, 58, 41, 56, 48, 55, 4...",[],[],"[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",w,0,1,0,0,False,False,False,False,False,False,0,True,True,True,True,0,0,,0,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[],[],64
1,0,64,D,e4,e2e4,"[7, 24, 7, 22, 2, 19, 2, 17, 16, 24, 15, 23, 1...",[],"[53, 37]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",b,0,1,1,0,False,False,False,False,False,False,0,True,True,True,True,53,37,,6,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[20],[],63
2,0,64,D,c6,c7c6,"[63, 48, 63, 46, 63, 53, 62, 17, 62, 26, 62, 3...",[],"[11, 19]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 0, 6, 6, 6, 6, ...",w,0,2,2,0,False,False,False,False,False,False,0,True,True,True,True,11,19,,6,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 19, 12, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[10],[],62
3,0,64,D,d4,d2d4,"[7, 24, 7, 22, 4, 11, 4, 18, 4, 25, 2, 17, 16,...",[],"[52, 36]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 0, 6, 6, 6, 6, ...",b,0,2,3,0,False,False,False,False,False,False,0,True,True,True,True,52,36,,6,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 19, 12, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[19],[],61
4,0,64,D,d5,d7d5,"[63, 48, 63, 46, 63, 53, 62, 17, 62, 26, 62, 3...","[37, 28]","[12, 28]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 0, 0, 6, 6, 6, ...",w,0,3,4,0,False,False,False,False,False,False,0,True,True,True,True,12,28,,6,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 19, 28, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[11],[],60


In [ ]:
moves = concat_shifted_features(moves)
moves.head(5)

,game_index,total_len,final_result,latest_move_san,latest_move_uci,possible_moves,possible_captures,latest_move,piece_placement,who_moves_next,halfmove_clock,fullmove_number,ply_counter,state_repetition,lm_capture,lm_check,lm_mate,lm_kingside_castling,lm_queenside_castling,lm_promotion,en_passant_target,K_castling_ability,Q_castling_ability,k_castling_ability,q_castling_ability,place_from,place_to,promoted_to,lm_piece_type,en_passant_target_prev,lm_en_passant,piece_tracking,piece_typing,lm_idx_moved,lm_idx_captured,plies_till_end,future_idx_moved,future_idx_captured,future_move,future_give_check_in_1,future_give_check_in_3,future_give_check_in_5,future_get_checked_in_1,future_get_checked_in_3,future_get_checked_in_5
0,0,64,D,--,----,"[63, 48, 63, 46, 58, 43, 58, 41, 56, 48, 55, 4...",[],[],"[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",w,0,1,0,0,False,False,False,False,False,False,0,True,True,True,True,0,0,,0,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[],[],64,"[20, 10, 19, 11, 20, 2, 30, 12, 18, 1]",[],"[53, 37]",False,False,False,False,False,False
1,0,64,D,e4,e2e4,"[7, 24, 7, 22, 2, 19, 2, 17, 16, 24, 15, 23, 1...",[],"[53, 37]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",b,0,1,1,0,False,False,False,False,False,False,0,True,True,True,True,53,37,,6,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[20],[],63,"[10, 19, 11, 20, 2, 30, 12, 18, 1, 29]",[],"[11, 19]",False,False,False,False,False,False
2,0,64,D,c6,c7c6,"[63, 48, 63, 46, 63, 53, 62, 17, 62, 26, 62, 3...",[],"[11, 19]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 0, 6, 6, 6, 6, ...",w,0,2,2,0,False,False,False,False,False,False,0,True,True,True,True,11,19,,6,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 19, 12, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[10],[],62,"[19, 11, 20, 2, 30, 12, 18, 1, 29, 13]",[],"[52, 36]",False,False,False,False,False,False
3,0,64,D,d4,d2d4,"[7, 24, 7, 22, 4, 11, 4, 18, 4, 25, 2, 17, 16,...",[],"[52, 36]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 0, 6, 6, 6, 6, ...",b,0,2,3,0,False,False,False,False,False,False,0,True,True,True,True,52,36,,6,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 19, 12, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[19],[],61,"[11, 20, 2, 30, 12, 18, 1, 29, 13, 28, 31]",[],"[12, 28]",False,False,False,False,False,False
4,0,64,D,d5,d7d5,"[63, 48, 63, 46, 63, 53, 62, 17, 62, 26, 62, 3...","[37, 28]","[12, 28]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 0, 0, 6, 6, 6, ...",w,0,3,4,0,False,False,False,False,False,False,0,True,True,True,True,12,28,,6,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 19, 28, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[11],[],60,"[20, 2, 30, 12, 18, 1, 29, 13, 28, 31, 13]",[20],"[37, 29]",False,False,False,False,False,False


In [ ]:
games_consistency_check(moves)

Iteration 64465
All tested passed successfully


In [ ]:
moves

,game_index,total_len,final_result,latest_move_san,latest_move_uci,possible_moves,possible_captures,latest_move,piece_placement,who_moves_next,halfmove_clock,fullmove_number,ply_counter,state_repetition,lm_capture,lm_check,lm_mate,lm_kingside_castling,lm_queenside_castling,lm_promotion,en_passant_target,K_castling_ability,Q_castling_ability,k_castling_ability,q_castling_ability,place_from,place_to,promoted_to,lm_piece_type,en_passant_target_prev,lm_en_passant,piece_tracking,piece_typing,lm_idx_moved,lm_idx_captured,plies_till_end,future_idx_moved,future_idx_captured,future_move,future_give_check_in_1,future_give_check_in_3,future_give_check_in_5,future_get_checked_in_1,future_get_checked_in_3,future_get_checked_in_5
0,0,64,D,--,----,"[63, 48, 63, 46, 58, 43, 58, 41, 56, 48, 55, 4...",[],[],"[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",w,0,1,0,0,False,False,False,False,False,False,0,True,True,True,True,0,0,,0,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[],[],64,"[20, 10, 19, 11, 20, 2, 30, 12, 18, 1]",[],"[53, 37]",False,False,False,False,False,False
1,0,64,D,e4,e2e4,"[7, 24, 7, 22, 2, 19, 2, 17, 16, 24, 15, 23, 1...",[],"[53, 37]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",b,0,1,1,0,False,False,False,False,False,False,0,True,True,True,True,53,37,,6,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[20],[],63,"[10, 19, 11, 20, 2, 30, 12, 18, 1, 29]",[],"[11, 19]",False,False,False,False,False,False
2,0,64,D,c6,c7c6,"[63, 48, 63, 46, 63, 53, 62, 17, 62, 26, 62, 3...",[],"[11, 19]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 0, 6, 6, 6, 6, ...",w,0,2,2,0,False,False,False,False,False,False,0,True,True,True,True,11,19,,6,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 19, 12, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[10],[],62,"[19, 11, 20, 2, 30, 12, 18, 1, 29, 13]",[],"[52, 36]",False,False,False,False,False,False
3,0,64,D,d4,d2d4,"[7, 24, 7, 22, 4, 11, 4, 18, 4, 25, 2, 17, 16,...",[],"[52, 36]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 0, 6, 6, 6, 6, ...",b,0,2,3,0,False,False,False,False,False,False,0,True,True,True,True,52,36,,6,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 19, 12, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[19],[],61,"[11, 20, 2, 30, 12, 18, 1, 29, 13, 28, 31]",[],"[12, 28]",False,False,False,False,False,False
4,0,64,D,d5,d7d5,"[63, 48, 63, 46, 63, 53, 62, 17, 62, 26, 62, 3...","[37, 28]","[12, 28]","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 0, 0, 6, 6, 6, ...",w,0,3,4,0,False,False,False,False,False,False,0,True,True,True,True,12,28,,6,0,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 19, 28, 13, 14...","[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, ...",[11],[],60,"[20, 2, 30, 12, 18, 1, 29, 13, 28, 31, 13]",[20],"[37, 29]",False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64460,499,159,D,Re1,b1e1,"[15, 8, 15, 7, 15, 6, 15, 16, 15, 14, 15, 24, ...","[36, 39]","[58, 61]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, ...",b,16,78,155,0,False,False,False,False,False,False,0,False,False,False,False,58,61,,1,0,False,"[36, 0, 0, 0, 15, 0, 0, 0, 0, 0, 0, 0, 21, 0, ...","[1, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 6, 0, 6, ...",[24],[],4,"[0, 28, 0, 28]",[],"[36, 44]",True,True,True,False,False,False
64461,499,159,D,Rd3+,d4d3,"[46, 38, 46, 37, 46, 55, 46, 54, 46, 53, 61, 45]",[],"[36, 44]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, ...",w,17,79,156,0,False,True,False,False,False,False,0,False,False,False,False,36,44,,1,0,False,"[44, 0, 0, 0, 15, 0, 0, 0, 0, 0, 0, 0, 21, 0, ...","[1, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 6, 0, 6, ...",[0],[],3,"[28, 0, 28]",[],"[46, 37]",False,False,False,False,False,False
64462,499,159,D,Ke4,f3e4,"[15, 8, 15, 7, 15, 6, 15, 16, 15, 14, 15, 24, ...",[],"[46, 37]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Unified function

In [ ]:
moves = generate_boards_daraframe(games_df=df_ccrl, n_from=2500, n_to=3000)

[2500-3000] Generating board features
> Parsings plays and replaying
Game: 3000
> Processing gather metadata into dataframe
> Tracking pieces
Game: 3000
> Appending initial position
> Appending shifted features
> Done!


In [ ]:
games_consistency_check(moves)

Iteration 66838
All tested passed successfully


In [ ]:
moves.to_parquet("boards_test.parquet")

# Generate boards

### 1 of 4

In [ ]:
!mkdir -p boards_ccrl/

for i in range(0, 150000, 2500):
    moves = generate_boards_daraframe(games_df=df_ccrl, n_from=i, n_to=i+2500)
    moves.to_parquet(f"boards_ccrl/boards_{i :0>7}.parquet")
    print()

[0-2500] Generating board features
> Parsings plays and replaying
Game: 2500
> Processing gather metadata into dataframe
> Tracking pieces
Game: 2500
> Appending initial position
> Appending shifted features
> Done!

[2500-5000] Generating board features
> Parsings plays and replaying
Game: 5000
> Processing gather metadata into dataframe
> Tracking pieces
Game: 5000
> Appending initial position
> Appending shifted features
> Done!

[5000-7500] Generating board features
> Parsings plays and replaying
Game: 7500
> Processing gather metadata into dataframe
> Tracking pieces
Game: 7500
> Appending initial position
> Appending shifted features
> Done!

[7500-10000] Generating board features
> Parsings plays and replaying
Game: 10000
> Processing gather metadata into dataframe
> Tracking pieces
Game: 10000
> Appending initial position
> Appending shifted features
> Done!

[10000-12500] Generating board features
> Parsings plays and replaying
Game: 12500
> Processing gather metadata into dat

In [ ]:
!du -sh boards_ccrl/

1.3G	boards_ccrl/


In [ ]:
!tar -cf boards_ccrl_1of4.tar boards_ccrl/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r boards_ccrl_1of4.tar "/content/drive/MyDrive/Praca magisterska/Colabs 4 - Final work/Data"

In [ ]:
drive.flush_and_unmount()

### 2 of 4

In [ ]:
!mkdir -p boards_ccrl/

for i in range(150000, 300000, 2500):
    moves = generate_boards_daraframe(games_df=df_ccrl, n_from=i, n_to=i+2500)
    moves.to_parquet(f"boards_ccrl/boards_{i :0>7}.parquet")
    print()

[150000-152500] Generating board features
> Parsings plays and replaying
Game: 152500
> Processing gather metadata into dataframe
> Tracking pieces
Game: 152500
> Appending initial position
> Appending shifted features
> Done!

[152500-155000] Generating board features
> Parsings plays and replaying
Game: 155000
> Processing gather metadata into dataframe
> Tracking pieces
Game: 155000
> Appending initial position
> Appending shifted features
> Done!

[155000-157500] Generating board features
> Parsings plays and replaying
Game: 157500
> Processing gather metadata into dataframe
> Tracking pieces
Game: 157500
> Appending initial position
> Appending shifted features
> Done!

[157500-160000] Generating board features
> Parsings plays and replaying
Game: 160000
> Processing gather metadata into dataframe
> Tracking pieces
Game: 160000
> Appending initial position
> Appending shifted features
> Done!

[160000-162500] Generating board features
> Parsings plays and replaying
Game: 162500
> 

In [ ]:
!du -sh boards_ccrl/

1.3G	boards_ccrl/


In [ ]:
!tar -cf boards_ccrl_2of4.tar boards_ccrl/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r boards_ccrl_2of4.tar "/content/drive/MyDrive/Praca magisterska/Colabs 4 - Final work/Data"

In [ ]:
drive.flush_and_unmount()

### 3 of 4

In [ ]:
!mkdir -p boards_ccrl/

for i in range(300000, 450000, 2500):
    moves = generate_boards_daraframe(games_df=df_ccrl, n_from=i, n_to=i+2500)
    moves.to_parquet(f"boards_ccrl/boards_{i :0>7}.parquet")
    print()

[300000-302500] Generating board features
> Parsings plays and replaying
Game: 302500
> Processing gather metadata into dataframe
> Tracking pieces
Game: 302500
> Appending initial position
> Appending shifted features
> Done!

[302500-305000] Generating board features
> Parsings plays and replaying
Game: 305000
> Processing gather metadata into dataframe
> Tracking pieces
Game: 305000
> Appending initial position
> Appending shifted features
> Done!

[305000-307500] Generating board features
> Parsings plays and replaying
Game: 307500
> Processing gather metadata into dataframe
> Tracking pieces
Game: 307500
> Appending initial position
> Appending shifted features
> Done!

[307500-310000] Generating board features
> Parsings plays and replaying
Game: 310000
> Processing gather metadata into dataframe
> Tracking pieces
Game: 310000
> Appending initial position
> Appending shifted features
> Done!

[310000-312500] Generating board features
> Parsings plays and replaying
Game: 312500
> 

In [ ]:
!du -sh boards_ccrl/

1.3G	boards_ccrl/


In [ ]:
!tar -cf boards_ccrl_3of4.tar boards_ccrl/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r boards_ccrl_3of4.tar "/content/drive/MyDrive/Praca magisterska/Colabs 4 - Final work/Data"

In [ ]:
drive.flush_and_unmount()

### 4 of 4

In [ ]:
!mkdir -p boards_ccrl/

for i in range(450000, 576220, 2500):
    moves = generate_boards_daraframe(games_df=df_ccrl, n_from=i, n_to=i+2500)
    moves.to_parquet(f"boards_ccrl/boards_{i :0>7}.parquet")
    print()

[450000-452500] Generating board features
> Parsings plays and replaying
Game: 452500
> Processing gather metadata into dataframe
> Tracking pieces
Game: 452500
> Appending initial position
> Appending shifted features
> Done!

[452500-455000] Generating board features
> Parsings plays and replaying
Game: 455000
> Processing gather metadata into dataframe
> Tracking pieces
Game: 455000
> Appending initial position
> Appending shifted features
> Done!

[455000-457500] Generating board features
> Parsings plays and replaying
Game: 457500
> Processing gather metadata into dataframe
> Tracking pieces
Game: 457500
> Appending initial position
> Appending shifted features
> Done!

[457500-460000] Generating board features
> Parsings plays and replaying
Game: 460000
> Processing gather metadata into dataframe
> Tracking pieces
Game: 460000
> Appending initial position
> Appending shifted features
> Done!

[460000-462500] Generating board features
> Parsings plays and replaying
Game: 462500
> 

In [ ]:
!du -sh boards_ccrl/

1.1G	boards_ccrl/


In [ ]:
!tar -cf boards_ccrl_4of4.tar boards_ccrl/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r boards_ccrl_4of4.tar "/content/drive/MyDrive/Praca magisterska/Colabs 4 - Final work/Data"

In [ ]:
drive.flush_and_unmount()